In [1]:
import pandas as pd
import numpy as np
import os
import shutil
import warnings
import pickle

# from mlxtend.preprocessing import TransactionEncoder
# from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth, fpmax

warnings.filterwarnings('ignore')

# 计算公式

### __1. TE = 6 * 10<sup>4</sup>* q<sub>p</sub> / (C<sub>p</sub> * V * T)__
* q<sub>p</sub>: 检测出的颗粒通量(个/L) — 150s
* C<sub>p</sub>: 数浓度(个/L) — 10<sup>9</sup>个
* V: 流速(ml/min) — 0.02
* T: 时间(s) — 150s

### __2. 某种同位素 _i_ 的数量 count<sub>i</sub> = count<sub>i</sub> / TE__
    


# 类的定义

In [2]:
class DataLoader:    # 原始数据预处理
    
    def __init__(self, ori_file_name):
        '''
        ori_file_name:原始数据文件的文件名。(原始数据放在当前目录下即可)
        '''
        self.ori_file_name = ori_file_name
        self.ori_df = pd.read_csv(ori_file_name).iloc[:, 2:]  # 原始数据的df (去掉index和timestamp)
        self.col_name = self.ori_df.columns.tolist()  # 完整粒子
        
        
            
    def get_ptc_name(self):
        '''
        返回剥离多余符号后的粒子名，如：46Ti
        '''
        short_name = list(map(lambda x: x[1:-8], self.col_name))
        return short_name

    
    
    def get_cleaned_data(self):
        '''
        清洗数据，将负值置为0, 返回清洗后数据的df
        '''
        cleaned_df = self.ori_df.copy()
        cleaned_df[cleaned_df <= 0] = np.nan  # 不大于0的区域全部置为nan
        return cleaned_df

    
    
    def get_basic_metirct(self, cleaned_df):
        '''
        得到 [去除符号后的粒子名，每种粒子出现的次数,最小强度，最大强度，平均强度，总强度，强度标准差] 的df；
        并插入metirc列作为index；最后返回基本指标的df
        cleaned_df：get_cleaned_data得到的清洗后数据的df
        '''
        basic_metric = pd.DataFrame()
        short_name = pd.DataFrame(np.array(self.get_ptc_name()).reshape(1, -1), columns=self.col_name)  # 去除符号后的粒子名
        count = cleaned_df[cleaned_df > 0].count().to_frame().T  # 某种粒子出现次数
        min_ints = cleaned_df.min().to_frame().T  # 某种粒子强度最小值
        max_ints = cleaned_df.max().to_frame().T  # 某种粒子强度最大值
        sum_ints = pd.DataFrame(np.nansum(cleaned_df, axis=0).reshape(1, -1), columns=self.col_name)  # 某种粒子强度和
        avg_ints = pd.DataFrame(np.nanmean(cleaned_df, axis=0).reshape(1, -1), columns=self.col_name)  # 某种粒子强度平均
        std_ints = pd.DataFrame(np.nanstd(cleaned_df, axis=0).reshape(1, -1), columns=self.col_name)  # 某种粒子强度的标准差
        basic_metric = pd.concat([short_name, min_ints, max_ints, count, sum_ints, avg_ints, std_ints], axis=0)
        basic_metric.insert(0, 'metric',
                            value=['ptc_name', 'min_ints', 'max_ints', 'count', 'sum_ints', 'avg_ints', 'std_ints'])
        basic_metric.set_index(['metric'], inplace=True)  # metric 列作为index
        return basic_metric

In [3]:
class PoissonMethod:    # 泊松分类法
    
    def __init__(self, data_df, metric_df, credible):
        '''
        data_df：清洗后数据的df
        metric_df：清洗后数据统计指标的df
        credible：置信度。小数表示，如0.997
        '''
        self.data_df = data_df
        self.metric_df = metric_df
        self.col_name = self.data_df.columns.tolist()
        self.m = self.metric_df.iloc[5]  # 未归一化的强度均值，归一化处理后作为λ
        self.credible = credible

        
        
    def normal_lambda(self):
        '''
        将强度均值归一化，得到可用于泊松计算λ。
        返回的df包括每种粒子的 [强度均值，λ，scale], 并返回该df，每行都是float。
        '''
        lamb_li = []  # 每种粒子归一化后的λ
        scale_li = []  # 每种粒子的缩放系数scale
        scale = 1.0
        for val in self.m:
            # 当k最大值为100时，概率累加到80时已超过1，因此平均强度归一化到80之内即可
            if val > 0 and val <= 1:
                scale = 80.0
            elif val > 1 and val <= 2:
                scale = 40.0
            elif val > 2 and val <= 3:
                scale = 30.0
            elif val > 3 and val <= 5:
                scale = 16.0
            elif val > 5 and val <= 10:
                scale = 8.0
            elif val > 10 and val <= 20:
                scale = 4.0
            elif val > 20 and val <= 40:
                scale = 2.0
            elif val > 40 and val <= 60:
                scale = 1.5
            elif val > 60 and val <= 80:
                scale = 1.0
            elif val > 80 and val <= 100:
                scale = 0.8
            elif val > 100 and val <= 200:
                scale = 0.4
            elif val > 200 and val <= 300:
                scale = 0.3
            elif val > 300 and val <= 400:
                scale = 0.2
            elif val > 400 and val <= 500:
                scale = 0.16
            elif val > 500 and val <= 800:
                scale = 0.1
            else:
                scale = 0.04

            lamb_li.append(round(val * scale))
            scale_li.append(scale)
            
        lamb_li = np.array(lamb_li).reshape(1, -1)
        scale_li = np.array(scale_li).reshape(1, -1)
        res_arr = np.concatenate((lamb_li, scale_li), axis=0)
        res_df = pd.DataFrame(res_arr, columns=self.col_name)
        res_df = pd.concat([self.m.to_frame().T, res_df])
        res_df.insert(0, 'metric', value=['avg_ints', 'lambda', 'scale'])
        res_df.set_index(['metric'], inplace=True)  # metric 列作为index
        return res_df

    
    
    def poisson(self, k, lamb):
        '''
        泊松方程，计算得到单词的概率值。在计算最终阈值时需要将概率累加
        lamb：归一化后的λ,一定是整数
        '''
        kjie = 1  # k!
        for i in range(1, k):
            kjie *= i
        lamb = float(lamb)
        pk = np.power(lamb, k) / kjie * np.exp(-lamb)
        return pk

    
    
    def get_ints_thr(self):
        '''
        计算得到每种元素的阈值df并返回。
        '''
        lamb = self.normal_lambda().iloc[1].values.astype('int')
        scale = self.normal_lambda().iloc[2].values
        ints_val = []
        for i in range(len(self.col_name)):
            thr = 0.0
            prob = 0.0
            for k in range(1, 100):
                prob += self.poisson(k, lamb[i])
                if prob >= self.credible:
                    thr = k / scale[i]
                    break
            ints_val.append(thr)
        ints_val = pd.DataFrame(np.array(ints_val).reshape(1, -1), columns=self.col_name)
        return ints_val

    
    
    def classifier(self):
        '''
        根据每种元素强度的阈值区分颗粒态和溶解态粒子，并返回颗粒态和溶解态分类结果的df。
        '''
        resolve = pd.DataFrame()  # 分类后的溶解态粒子数据
        particle = pd.DataFrame()  # 分类后的颗粒态粒子数据
        ints_thr = self.get_ints_thr()
        ints_thr_li = ints_thr.values[0]

        for idx in range(len(self.col_name)):
            single_ptc_df = self.data_df.iloc[:, idx].to_frame()
            single_ptc_resolve = single_ptc_df[single_ptc_df >= ints_thr_li[idx]]
            particle = pd.concat([particle, single_ptc_resolve], axis=1)

        resolve = self.data_df[pd.isnull(particle)]
        return particle, resolve

In [4]:
class IterMethod:    # 迭代分类法

    def __init__(self, data_df, metric_df):
        '''
        data_csv: 清洗后的df
        metric_csv: 清洗后数据统计指标的df
        '''
        self.data_df = data_df
        self.metric_df = metric_df
        self.col_name = self.data_df.columns
        self.cur_df = self.data_df  # 每次要进行被迭代的df
        self.iter_cnt = 0  # 迭代次数


        
    def get_avg(self):
        '''
        求出每种粒子强度的平均值，返回nparray
        '''
        return np.nanmean(self.cur_df, axis=0)

    
    
    def get_std(self):
        '''
        求出每种粒子强度的标准差，返回nparray
        '''
        return np.nanstd(self.cur_df, axis=0)

    
    
    def get_thr(self):
        '''
        求出每个csv文件的阈值并返回对应df,并更新self.iter_cnt。
        '''
        avg_tmp = self.get_avg()
        std_tmp = self.get_std()
        thr = 3.29 * std_tmp + 2.71 + avg_tmp
        thr_df = pd.DataFrame([thr] * len(self.data_df), columns=self.col_name).fillna(0)
        self.iter_cnt += 1
        return thr_df

    
    
    def gt_file(self, cnt):
        '''
        创建gt文件夹并返回对应迭代轮数的csv文件名
        '''
        dir_path = os.path.join(os.getcwd(), "gt")
        flag = os.path.exists(dir_path)
        if not flag:
            os.makedirs(dir_path)
        file_name = str(cnt) + ".csv"
        file_path = os.path.join(dir_path, file_name)
        return file_path

    
    
    def lt_file(self, cnt):
        '''
        创建lt文件夹并返回对应迭代轮数的csv文件名, 如：gt/1.csv
        '''
        dir_path = os.path.join(os.getcwd(), "lt")
        flag = os.path.exists(dir_path)
        if not flag:
            os.makedirs(dir_path)
        file_name = str(cnt) + ".csv"
        file_path = os.path.join(dir_path, file_name)
        return file_path

    
    
    def update_df(self, thr, cnt):
        '''
        根据阈值保存为两个csv文件：'gt/1.csv'、'lt/1.csv'；并返回小于阈值的df
        '''
        gt_df = self.cur_df[self.cur_df >= thr]
        gt_df.to_csv(self.gt_file(cnt), index=None)
        lt_df = self.cur_df[self.cur_df < thr]
        lt_df.to_csv(self.lt_file(cnt), index=None)
        return lt_df

    
    
    def iterator(self):
        '''
        迭代过程
        '''
        beg_DF = self.cur_df
        end_DF = self.cur_df
        flag = False
        while not flag:
            beg_DF = end_DF
            THR = self.get_thr()  # self.iter_cnt 在此处已+1
            end_DF = self.update_df(THR, self.iter_cnt)
            self.cur_df = end_DF
            if beg_DF.equals(end_DF):
                flag = True
                print("Iteration have been finished, cnt: %s." % self.iter_cnt)


                
    def get_final_result(self):
        '''
        根据溶解态的结果，得到颗粒态的最终结果。并返回两个df
        '''
        res_df = pd.read_csv('./lt/'+str(self.iter_cnt)+'.csv')
        ptc_df = self.data_df[pd.isnull(res_df)]
        # pd.isnull(resolve_df_：溶解态df的非空位为False，空位为True，与清洗后的原始数据做mask
        ptc_df.to_csv('./gt/'+str(self.iter_cnt+1)+'.csv', index=None)
        return ptc_df, res_df

In [5]:
class BackgroudConcentration:    # 减背景&计算颗粒数浓度
    
    def __init__(self, ptc_df, res_df):
        '''
        ptc_df ：颗粒态数据df
        res_df：溶解态数据df
        '''
        self.ptc_df = ptc_df
        self.res_df = res_df
        self.df_len = len(self.res_df)
        self.col_name = self.ptc_df.columns

        
        
    def get_background(self):
        '''
        计算每种同位素的背景值并返回对应df。
        '''
        BG = pd.DataFrame([np.nanmean(self.res_df, axis=0)] * self.df_len, columns=self.col_name).fillna(0)
        return BG

    
    
    def substract_background(self):
        '''
        对颗粒态数据减去背景值并返回减背景后的df,同时保存csv
        background_df：背景值df
        '''
        BG = self.get_background()
        substract_bg_particle = self.ptc_df - BG
        file_name = 'particle_classified_final.csv'
        substract_bg_particle.to_csv(file_name, index=None)
        print("Particle final result, whose background have been substracted.")
        return substract_bg_particle
        
        
    def select_columns(self, target_particle):
        '''
        在减去背景的颗粒态数据中选择要处理的同位素，组成df并返回
        target_particle：要选择的粒子名列表，如:['27Al','206Pb']。手动输入
        '''
        ptc_df = self.substract_background()
        ptc_name_full_li = ptc_df.columns.tolist()  # 表头
        ptc_name_short_li = list(map(lambda x: x[1:-8], ptc_name_full_li))  # 粒子名：原子质量+元素名
        select_col_li = []  # select_col_li 选中元素所在列的完整列名

        for item in target_particle:
            for i in range(len(ptc_name_full_li)):
                if item == ptc_name_short_li[i]:
                    select_col_li.append(ptc_name_full_li[i])

        selected_ptc_df = ptc_df[select_col_li]
        return selected_ptc_df
        
        

      
    def get_TE(self, C, V, T):
        '''
        利用减去背景值后的只包含197Au的df，计算得到TE，并保存对应csv。
        C:数浓度(个/L)
        V:流速(ml/min)
        T:时间(s)
        '''
        te_df = self.substract_background()
        ptc_cnt = te_df.count()
        coef = 6e4 / (C * V * T)
        TE = pd.DataFrame(ptc_cnt * coef, columns=te_df.columns)
        TE.to_csv("TE.csv", index=None)
        print("TE have been computed.")

        
        
    def get_isotopes_number(self, selected_ptc_df, TE):
        '''
        计算去除背景后颗粒态的目标同位素的颗粒数。
        selected_ptc_df：筛选出的要计算数量的颗粒态粒子df
        TE：计算参数，手动输入
        '''
        ptc_cnt = selected_ptc_df.count()
        res = ptc_cnt / TE
        ptc_num_con = res.to_frame().T
        file_name = "number.csv"
        ptc_num_con.to_csv(file_name, index=None)
        print("Particle number have been computed.")

# 完整执行流程

In [10]:
def main():
    # Step1：为当前文件创建保存生成文件的文件夹，并切换到对应文件夹路径下。
    cls_method  = '_iteration' if iteration else '_poisson'    #　文件夹后缀。分类方法不同导致创建的文件夹后缀不同
    dir_name = origin_csv[:-4] + cls_method
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
    print('Directory of %s have been created!' % dir_name)
    
    shutil.copyfile(origin_csv, dir_name+'/'+origin_csv)   # 复制原文件到目标文件夹
    os.chdir(dir_name)   # 切换到保存生成文件的目录
    print('cwd: %s' % os.getcwd()) # 查看当前目录
    
    
    
    # Step2：执行分类、减背景、(计算TE)和颗粒数浓度
    # 默认泊松法
    if not iteration:
        #  一：预处理
        data_loader = DataLoader(origin_csv)  # 实例化
        cleaned_data = data_loader.get_cleaned_data()  # 得到清洗后的数据
        metric_data = data_loader.get_basic_metirct(cleaned_data)  # 得到相关指标统计结果
        # 二：Poisson执行
        poissonmethod = PoissonMethod(cleaned_data, metric_data, credible)  # 实例化
        ptc_df, res_df = poissonmethod.classifier()  # 分类得到颗粒态和溶解态数据csv

    #　迭代法
    else:
        #  一：预处理
        data_loader = DataLoader(origin_csv)  # 实例化
        cleaned_data = data_loader.get_cleaned_data()  # 得到清洗后的数据
        metric_data = data_loader.get_basic_metirct(cleaned_data)  # 得到相关指标统计结果
        # 二：Iteration执行
        itermethod = IterMethod(cleaned_data, metric_data)     # 实例化
        itermethod.iterator()   # 迭代过程
        ptc_df, res_df = itermethod.get_final_result()   # 得到最终的颗粒态数据，将最终的文件复制并返回文件名

        
        
    # Step3：减背景、颗粒数浓度
    BG_C = BackgroudConcentration(ptc_df, res_df)   # 实例化
    
    if TE_flag:
        BG_C.get_TE(C, V, T)
        
    else:
        selected_ptc_df = BG_C.select_columns(isotopes_li)   # 选择要计算浓度的元素，以列表形式存放。同时保存颗粒态数据csv。
        BG_C.get_isotopes_number(selected_ptc_df, TE)         # 计算颗粒态数据的颗粒数浓度
    
    
    
    # Step4：返回上级目录
    print('-' * 50)
    os.chdir('../')

# 超参数

In [15]:
file_li = ['S1.csv',  'S19.csv', 'S91.csv']  
# file_li = ['S1_TE.csv', 'S19_TE.csv', 'S91_TE.csv'] 

for file in file_li:
    origin_csv = file  # 原文件(放在上一级目录，会自动copy)
    
    iteration = True   # 分类方法
    credible = 0.997   # 泊松法置信度
    
    TE_flag = False     # 是否计算TE
    C = 1e9            # 数浓度
    V = 0.02           # 流速
    T = 150            # 时间
    
    TE = 0.22402       # TE
    # 要计算颗粒数的粒子（需要人工筛选）   
    isotopes_li = ['24Mg','27Al', '42Ca', '44Ca', '46Ti', '47Ti', '48Ti', '49Ti', '50Ti', '51V', '52Cr', '54Fe', '55Mn', '57Fe', '58Fe',  '59Co','60Ni', '63Cu', '66Zn', 
                     '75As', '82Se', '86Sr', '87Sr','88Sr','89Y', '95Mo', '98Mo','107Ag', '111Cd','112Sn','121Sb', '112Sn', '121Sb','138Ba', '139La', '140Ce', '141Pr', '146Nd', '147Sm', '153Eu', '157Gd', '159Tb', 
                     '163Dy', '165Ho', '166Er', '169Tm', '172Yb', '175Lu', '197Au','205Tl', '206Pb', '207Pb', '208Pb']

    if __name__ == '__main__':
        main()

Directory of S1_iteration have been created!
cwd: C:\Users\yangyang.wang3\Desktop\最终版本_记得更新\S1_iteration
Iteration have been finished, cnt: 13.
Particle final result, whose background have been substracted.
Particle number have been computed.
--------------------------------------------------
Directory of S19_iteration have been created!
cwd: C:\Users\yangyang.wang3\Desktop\最终版本_记得更新\S19_iteration
Iteration have been finished, cnt: 18.
Particle final result, whose background have been substracted.
Particle number have been computed.
--------------------------------------------------
Directory of S91_iteration have been created!
cwd: C:\Users\yangyang.wang3\Desktop\最终版本_记得更新\S91_iteration
Iteration have been finished, cnt: 12.
Particle final result, whose background have been substracted.
Particle number have been computed.
--------------------------------------------------
